In [184]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [185]:
model = model = load_model('models/cnn_digits_full_dataset.keras')

In [186]:
image = cv2.imread("sudoku_extracted.png")

In [187]:
def split_cells(image):
    cells = []
    edge = 450 // 9  # 50 pixels per cell in 450x450 image
    for y in range(9):
        for x in range(9):
            x1, y1 = x * edge, y * edge
            cell = image[y1:y1+edge, x1:x1+edge]
            cells.append(cell)
    return cells

In [188]:
# image = cv2.imread("sudoku_extracted.png")
# cells = split_cells(image)

# for idx, cell in enumerate(cells):
#     cv2.imshow(f"Cell {idx}", cell)
#     print(f"Showing Cell {idx}")
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
# # The code above loads a pre-trained CNN model and splits a Sudoku puzzle image into its individual cells.
# # Each cell is displayed one by one in a window.

In [189]:
def preprocess_cell(cell):
    gray = cv2.cvtColor(cell, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
    
    black_pixels = thresh.shape[0] * thresh.shape[1] - cv2.countNonZero(thresh)
    if black_pixels < 50:
        return None


    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    digit_roi = thresh[y:y + h, x:x + w]

    padded = cv2.copyMakeBorder(digit_roi, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=255)
    resized = cv2.resize(padded, (28, 28))
    normalized = resized / 255.0
    reshaped = normalized.reshape(1, 28, 28, 1)
    return reshaped

In [190]:
# # We will see the cells one by one, preprocess them, and display the processed images.


# cells = split_cells(image)

# for idx, cell in enumerate(cells):
#     processed = preprocess_cell(cell)

#     if processed is not None:
#         digit_img = (processed[0] * 255).astype("uint8").reshape(28, 28)  # Undo normalization
#         cv2.imshow(f"Cell {idx}", digit_img)
#     else:
#         print(f"Cell {idx}: Empty or Unreadable")

#     key = cv2.waitKey(0)
#     if key == ord('q'):
#         break

# cv2.destroyAllWindows()


In [ ]:
def recognize_digits(image_path):
    board = np.zeros((9, 9), dtype=int)
    image = cv2.imread(image_path)
    cells = split_cells(image)

    for idx, cell in enumerate(cells):
        processed = preprocess_cell(cell)
        row, col = divmod(idx, 9)

        if processed is not None:
            pred = model.predict(processed, verbose=0)
            digit_class = np.argmax(pred)
            board[row, col] = digit_class  # Map class 0→digit 1, ..., class 8→digit 9
        else:
            continue
    return board


In [196]:
grid = recognize_digits("sudoku_extracted.png")
print(grid)

[[3 4 1 8 1 1 4 1 2]
 [2 7 1 6 3 6 6 1 7]
 [2 6 4 2 1 4 1 1 1]
 [8 1 9 4 1 3 1 1 9]
 [2 8 7 7 1 1 1 5 1]
 [7 7 2 4 1 3 4 1 8]
 [7 1 7 5 1 8 1 1 1]
 [8 4 4 6 8 7 7 1 5]
 [6 1 6 3 1 7 1 4 3]]


In [152]:
# #Testing on a single cell

# import cv2
# import numpy as np

# # Load image
# image = cv2.imread("sudoku_extracted.png")

# # Split cells
# cells = split_cells(image)

# # Select any one cell — for example, top-left cell (0, 0)
# cell = cells[3]

# # Preprocess it
# processed = preprocess_cell(cell)

# # Show the preprocessed cell
# if processed is not None:
#     cv2.imshow("Preprocessed Cell", processed.reshape(28, 28))
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

#     # Predict
#     pred = model.predict(processed, verbose=0)
#     digit = int(np.argmax(pred))
#     confidence = float(np.max(pred))

#     print("Predicted Digit:", digit)  # Adding 1 to match the digit (0-9) to (1-9)
#     print("Confidence:", round(confidence, 3))
# else:
#     print("Digit could not be detected in this cell.")
